[![The Data](https://img.shields.io/badge/The%20Data-Click%20Here-blue)](https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv)
[![The Analysis](https://img.shields.io/badge/The%20Analysis-Click%20Here-green)](https://github.com/mbabczak/mbabczak.github.io/tree/main/Lab)


# !!! For some reason, I am having issues with the graphs loading in this enviorment. To work around this, I downloaded them as HTML files that allow the grader to still interact with the plots. I will push both HTML to the github repo aswell. !!!

In [23]:
import altair as alt

In [34]:
import pandas as pd
alt.data_transformers.enable('json')
url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv"
df = pd.read_csv(url)
df_clean = df.dropna(subset=["Year Acquired", "Square Footage", "Agency Name", "Usage Description"])
df_clean["Year Acquired"] = pd.to_numeric(df_clean["Year Acquired"], errors="coerce").astype("Int64")
df_clean = df_clean[(df_clean["Year Acquired"] >= 1800) & (df_clean["Year Acquired"] <= 2025)]
df_clean


,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8851,Department of Transportation,Biggsville - Henderson County,Route 1,Biggsville,61418,Henderson,17,Cheri Bustos,94,Frese Randy E.,...,In Progress,2018,2018,1455,1,0,0,Storage,Storage,NaN
8852,Department of Transportation,Decatur Maintenance Facility - Macon County,4142 N. Westlawn Avenue,Decatur,62526,Macon,4,Luis Gutierrez,4,Soto Cynthia,...,In Progress,2017,2017,6720,1,0,0,Storage,Utiility & Miscellan,NaN
8853,Department of Transportation,Marion Maintenance Facility - Williamson County,Interstate 57 Milepost 46,Marion,62959,Williamson,12,Mike Bost,117,Severin Dave,...,In Progress,2019,2019,760,1,0,0,Business,Business,NaN
8854,Department of Transportation,Marion Maintenance Facility - Williamson County,Intersate 57 Milepost 46,Marion,62959,Williamson,12,Mike Bost,117,Severin Dave,...,In Progress,2019,2019,760,1,0,0,Business,Business,NaN


In [30]:
alt.data_transformers.disable_max_rows()
alt.renderers.enable('default')

# Chart 1
agency_counts = df_clean.groupby("Agency Name").size().reset_index(name="Count")
chart1 = (
    alt.Chart(agency_counts)
    .mark_bar()
    .encode(
        x=alt.X("Count:Q", title="Number of Buildings"),
        y=alt.Y(
            "Agency Name:N",
            sort=alt.SortField(field="Count", order="descending"),  # simpler sort
            title="Agency"
        ),
        color=alt.Color("Count:Q", scale=alt.Scale(scheme="blues"), legend=None),
        tooltip=["Agency Name", "Count"]
    )
    .properties(
        title="Number of Illinois Buildings Owned by Each Agency",
        width=600,
        height=600  
    )
)
chart1
chart1.save("chart1.html")


In [36]:
chart1.save("chart1.json")


# Chart 1 HTML: file:///Users/mollybabczak/Desktop/mbabczak.github.io/Lab/chart1.html

For my first plot, I visualized the number of Illinois buildings owned by each state agency. The x-axis represents the number of buildings, and the y-axis lists the agencies that own them. I included a descriptive title to ensure the viewer immediately understands what the chart conveys. To make the visualization easier to interpret, I sorted the y-axis in descending order so it’s clear which agencies own the most and fewest buildings at a glance. Before creating the plot, I used a Pandas groupby function to group the data by agency and count the number of buildings per agency. This preprocessing step simplified the data and made it straightforward to display the results clearly in the bar chart. It was very interesting to see that the department of natural resources was the largest agency by far.

In [35]:
# Chart 2
agency_dropdown = alt.binding_select(options=sorted(df_clean["Agency Name"].unique()), name="Select Agency: ")
agency_select = alt.selection_point(fields=["Agency Name"], bind=agency_dropdown)

chart2 = (
    alt.Chart(df_clean)
    .mark_circle(size=60, opacity=0.7)
    .encode(
        x=alt.X("Year Acquired:Q", title="Year Acquired", scale=alt.Scale(domain=[1800, 2025])),
        y=alt.Y("Square Footage:Q", title="Building Size (Sq. Ft)"),
        color=alt.Color("Usage Description:N", title="Usage Type", scale=alt.Scale(scheme="tableau10")),
        tooltip=["Agency Name", "Square Footage", "Year Acquired", "Usage Description"]
    )
    .add_params(agency_select)
    .transform_filter(agency_select)
    .properties(
        title="Building Size vs. Year Acquired by Agency",
        width=600,
        height=400
    )
)

chart2.save("chart2.html")


In [37]:
chart2.save("chart2.json")


# Chart 2 HTML: file:///Users/mollybabczak/Desktop/mbabczak.github.io/Lab/chart2.html

For my second visualization, I decided to create an interactive scatter plot that to visualize the relationship between the square footage of a building and the year it was acquired, filtered by the agency that owns them. On the X-axis, I put the year it was acquired and on the y-axis, I put the building square footage. The interactive element is that the user can select the agency that they want to view and they will only see buildings that the selected agency acquired. 

To allow the user to distinguish between building types, I decided to make a different color dot represent the different types of buildings to allow the user to see the distribution of building types. An issue that I encountered while creating this visual was with the year acquired. When I had no conditions on the year, the X-axis would showed all years all the way back to year 0. This made the visual hard to interpret and difficult to see the individual dots. Upon further examination, I saw that there were 256 rows where their year acquired was the year 0. For the purpose of my visual, I decided to cap the years shown to 1800, since there were no years lower than that besides the ones showing 0. I decided to not include those values because it seems like there was either a value missing or the building was not accurately colllected. 


I found this to be interesting, especially for the University of Illinois. It was really cool to explore when different buildings were acquired. Something that I had noticed that caused me to worry was there was a residential building that U of I acquired in 1857. This didn't make sense to me, since the University was founded in 1867. However, I realized that this was probably because both the Chicago and Springfield campuses were being included. 